In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
!pip install transformers

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes

In [5]:
!pip install sentencepiece
!pip install langchain_community

  Using cached langchain-0.2.14-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_core-0.2.34-py3-none-any.whl.metadata (6.2 kB)
  Using cached langsmith-0.1.101-py3-none-any.whl.metadata (13 kB)
  Using cached langchain_text_splitters-0.2.2-py3-none-any.whl.metadata (2.1 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached httpx-0.27.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached httpcore-1.0.5-py3-none-any.whl.metadata (20 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 12.0 MB/s eta 0:00:00
Using cached langchain-0.2.14-py3-none-any.whl (997 kB)
Using cached langchain_core-0.2.34-py3-none-any.whl (393 kB)
Using cached langsmith-0.1.101-py3-none-any.whl (148 kB)
Using cached httpx-0.27.0-py3-none-any.whl (75 kB)
Using cached httpcore-1.0.5-py3-none-any.whl (77 kB)
Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
Using cached langchain_text_splitters-0.2.2-py3-none-any.whl (25 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline
import torch

from transformers import LlamaTokenizer, LlamaForCausalLM

model_path = "openlm-research/open_llama_7b"

tokenizer = LlamaTokenizer.from_pretrained(model_path)
model = LlamaForCausalLM.from_pretrained(
    model_path, torch_dtype=torch.float16, device_map='auto',
)

pipeline = pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

In [ ]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

In [1]:
json_data = {
    "patient_name": "Mr. Abc",
    "referring_doctor": "Dr. Xyz",
    "compliant": "Slurred speech and Weakness",
    "diagnosis": "ACUTE ISCHEMIC STROKE",
    "sessions_done": [
        {
            "date": "15-Jul-24",
            "name": "Strength Training",
            "count": 2,
            "activities": [
                {"name": "Weight Cuff", "level": "Level 1"},
                {"name": "Swiss Ball", "level": "Level 1"},
                {"name": "Erogmeter", "level": "Level 1"}
            ]
        }
    ],
    "improvements": [
        {"date": "15-Jul-24", "improvement": "Foleys Catheter Removal"},
        {"date": "18-Jul-24", "improvement": "Sitting with support"},
        {"date": "21-Jul-24", "improvement": "Wheel chair mobilisation"}
    ],
    "milestones": "Tracstomy Tube Removed, Sitting without support",
    "vitals_admission": {
        "heart_rate": "70",
        "blood_pressure": "120/80",
        "temperature": "98.6",
        "respiratory_rate": 16
    },
    "vital_last_week": {
        "heart_rate": "80",
        "blood_pressure": "130/85",
        "temperature": "99.5",
        "respiratory_rate": 18
    },
    "vital_weekly": [
        {"date": "15-Jul-24", "heart_rate": "70", "blood_pressure": "120/80", "temperature": "98.6", "respiratory_rate": 16}
    ],
    "procedures_done": [
        {"date": "15-Jul-24", "name": "Wound Dressings"},
        {"date": "17-Jul-24", "name": "Tube Changing(NG,FC)"},
        {"date": "19-Jul-24", "name": "Nebulization"}
    ]
}

# Define the prompt for summarization
'''prompt = """You are a JSON data summarizer. You will be taking the JSON data and
summarizing it into a paragraph format, outputting it in 10 lines."""'''

# Function to preprocess JSON data
def preprocess_json(json_data):
    """Preprocess JSON data into a formatted string for summarization."""
    patient_name = json_data.get('patient_name', 'Unknown')
    referring_doctor = json_data.get('referring_doctor', 'Unknown')
    complaint = json_data.get('compliant', 'Not specified')
    diagnosis = json_data.get('diagnosis', 'Not specified')

    # Treatment Sessions
    sessions = json_data.get('sessions_done', [])
    session_details = ""
    for session in sessions:
        date = session.get('date', 'Unknown date')
        name = session.get('name', 'Unknown')
        activities = session.get('activities', [])
        activities_str = ', '.join([f"{activity['name']} at {activity['level']}" for activity in activities])
        session_details += f"On {date}, {name} included activities such as {activities_str}. "

    # Key Improvements
    improvements = json_data.get('improvements', [])
    improvements_str = ', '.join([f"{imp['improvement']} on {imp['date']}" for imp in improvements])

    # Milestones
    milestones = json_data.get('milestones', 'Not specified')

    # Vitals
    vitals_admission = json_data.get('vitals_admission', {})
    recent_vitals = json_data.get('vital_last_week', {})
    vital_details = (
        f"Vital Signs on Admission: Heart Rate: {vitals_admission.get('heart_rate', 'N/A')}, "
        f"Blood Pressure: {vitals_admission.get('blood_pressure', 'N/A')}, "
        f"Temperature: {vitals_admission.get('temperature', 'N/A')}, "
        f"Respiratory Rate: {vitals_admission.get('respiratory_rate', 'N/A')}. "
        f"Recent Vital Signs: Heart Rate: {recent_vitals.get('heart_rate', 'N/A')}, "
        f"Blood Pressure: {recent_vitals.get('blood_pressure', 'N/A')}, "
        f"Temperature: {recent_vitals.get('temperature', 'N/A')}, "
        f"Respiratory Rate: {recent_vitals.get('respiratory_rate', 'N/A')}. "
    )

    # Procedures Done
    procedures = json_data.get('procedures_done', [])
    procedures_str = ', '.join([f"{proc['name']} on {proc['date']}" for proc in procedures])

    return (
        f"Patient Name: {patient_name}. Referring Doctor: {referring_doctor}. "
        f"Complaint: {complaint}. Diagnosis: {diagnosis}. "
        f"Treatment Sessions: {session_details} "
        f"Key Improvements: {improvements_str}. "
        f"Milestones Achieved: {milestones}. "
        f"{vital_details} "
        f"Procedures Done: {procedures_str}."
    )


In [2]:
print(preprocess_json(json_data))

Patient Name: Mr. Abc. Referring Doctor: Dr. Xyz. Complaint: Slurred speech and Weakness. Diagnosis: ACUTE ISCHEMIC STROKE. Treatment Sessions: On 15-Jul-24, Strength Training included activities such as Weight Cuff at Level 1, Swiss Ball at Level 1, Erogmeter at Level 1.  Key Improvements: Foleys Catheter Removal on 15-Jul-24, Sitting with support on 18-Jul-24, Wheel chair mobilisation on 21-Jul-24. Milestones Achieved: Tracstomy Tube Removed, Sitting without support. Vital Signs on Admission: Heart Rate: 70, Blood Pressure: 120/80, Temperature: 98.6, Respiratory Rate: 16. Recent Vital Signs: Heart Rate: 80, Blood Pressure: 130/85, Temperature: 99.5, Respiratory Rate: 18.  Procedures Done: Wound Dressings on 15-Jul-24, Tube Changing(NG,FC) on 17-Jul-24, Nebulization on 19-Jul-24.


In [3]:
from langchain import PromptTemplate,  LLMChain

template = """
            Write a concise summary of the following text delimited by triple backquotes.
            Return your response in single paragraph which covers the key points of the text and make sure that the summary must be intelegent answer.
              ```{text}```
              SINGLE PARAGRAPH SUMMARY:
           """

prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain = LLMChain(prompt=prompt, llm=llm)

text = """
Patient Name: Mr. Abc. Referring Doctor: Dr. Xyz. Complaint: Slurred speech and Weakness. Diagnosis: ACUTE ISCHEMIC STROKE. Treatment Sessions: On 15-Jul-24, Strength Training included activities such as Weight Cuff at Level 1, Swiss Ball at Level 1, Erogmeter at Level 1.  Key Improvements: Foleys Catheter Removal on 15-Jul-24, Sitting with support on 18-Jul-24, Wheel chair mobilisation on 21-Jul-24. Milestones Achieved: Tracstomy Tube Removed, Sitting without support. Vital Signs on Admission: Heart Rate: 70, Blood Pressure: 120/80, Temperature: 98.6, Respiratory Rate: 16. Recent Vital Signs: Heart Rate: 80, Blood Pressure: 130/85, Temperature: 99.5, Respiratory Rate: 18.  Procedures Done: Wound Dressings on 15-Jul-24, Tube Changing(NG,FC) on 17-Jul-24, Nebulization on 19-Jul-24.


"""

print(llm_chain.run(text))

ModuleNotFoundError: No module named 'langchain'